<a href="https://colab.research.google.com/github/hc2twv/UPSE_OP/blob/main/ModCostOptim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 45.5 MB/s eta 0:00:00


En este código:

* Creación del problema: Se define un problema de optimización de minimización utilizando pulp.LpProblem.

* Variables de decisión: Se crean variables continuas x1, x2, x3 que podrían representar, por ejemplo, la cantidad de recursos asignados a diferentes nodos de la red.

* Función objetivo: Se define la función objetivo como una combinación lineal de las variables de decisión, representando los costos operativos.

* Restricciones: Se añaden varias restricciones lineales que podrían representar la capacidad mínima requerida entre diferentes nodos.

* Resolución del problema: Se utiliza el método solve para encontrar los valores óptimos de las variables que minimizan el costo mientras satisfacen las restricciones.

* Resultados: Se imprimen el estado de la solución, el costo total y los valores óptimos de las variables de decisión.1

In [3]:
import pulp

# Crear el problema de minimización
problem = pulp.LpProblem("Network_Optimization", pulp.LpMinimize)

# Definir las variables de decisión
# Ejemplo: cantidad de recursos a asignar en diferentes nodos de la red
x1 = pulp.LpVariable('x1', lowBound=0, cat='Continuous')
x2 = pulp.LpVariable('x2', lowBound=0, cat='Continuous')
x3 = pulp.LpVariable('x3', lowBound=0, cat='Continuous')

# Definir la función objetivo (minimizar costos operativos)
# Ejemplo: Costo total = costo_unitario_nodo1 * x1 + costo_unitario_nodo2 * x2 + costo_unitario_nodo3 * x3
cost = 5*x1 + 8*x2 + 3*x3
problem += cost, "Costo_Total"

# Definir las restricciones del problema
# Ejemplo: Restricciones de capacidad y rendimiento
# Capacidad de nodo1 + nodo2 >= 10
problem += x1 + x2 >= 10, "Capacidad_Nodo1_Nodo2"
# Capacidad de nodo2 + nodo3 >= 20
problem += x2 + x3 >= 20, "Capacidad_Nodo2_Nodo3"
# Capacidad de nodo1 + nodo3 >= 15
problem += x1 + x3 >= 15, "Capacidad_Nodo1_Nodo3"

# Resolver el problema
problem.solve()

# Imprimir los resultados
print("Estado:", pulp.LpStatus[problem.status])
print("Costo Total:", pulp.value(problem.objective))
print("x1:", pulp.value(x1))
print("x2:", pulp.value(x2))
print("x3:", pulp.value(x3))


Estado: Optimal
Costo Total: 110.0
x1: 10.0
x2: 0.0
x3: 20.0


Un ejemplo más complejo en el campo de las telecomunicaciones. Supondremos que estamos gestionando una red de telecomunicaciones con varios nodos, enlaces y tráfico que debe ser enrutado a través de la red. Queremos minimizar los costos operativos y de transmisión mientras satisfacemos las demandas de tráfico y las capacidades de los enlaces.

En este código:

* Definición del problema: Se define un problema de minimización utilizando pulp.LpProblem.

* Datos del problema: Se definen los nodos, enlaces (con capacidad y costo), y las demandas de tráfico entre nodos.

* Variables de decisión: Se crean variables continuas x[(i, j)] que representan el flujo de tráfico en cada enlace.

* Función objetivo: Se define la función objetivo como la suma de los productos del flujo de tráfico en cada enlace y su costo, con el objetivo de minimizar esta suma.

* Restricciones:

  * Capacidad de los enlaces: Asegura que el flujo en cada enlace no supere su capacidad.
  * Balance de flujo en los nodos: Asegura que el flujo entrante y saliente en cada nodo sea equilibrado según las demandas de tráfico.

* Resolución del problema: Se utiliza el método solve para encontrar los valores óptimos de las variables que minimizan el costo mientras satisfacen las restricciones.

* Resultados: Se imprimen el estado de la solución, el costo total y los flujos de tráfico en cada enlace.

In [10]:
import pulp

# Crear el problema de minimización
problem = pulp.LpProblem("Telecom_Network_Optimization", pulp.LpMinimize)

# Definir los datos del problema
# Nodos
nodos = ['A', 'B', 'C', 'D']

# Enlaces (de, a) con capacidad y costo
enlaces = {
    ('A', 'B'): {'capacidad': 15, 'costo': 2},
    ('A', 'C'): {'capacidad': 10, 'costo': 4},
    ('B', 'C'): {'capacidad': 20, 'costo': 1},
    ('B', 'D'): {'capacidad': 10, 'costo': 3},
    ('C', 'D'): {'capacidad': 25, 'costo': 2}
}

# Demandas de tráfico (de, a) y la cantidad de tráfico
demandas = {
    ('A', 'D'): 10,
    ('B', 'D'): 15,
    ('A', 'C'): 5
}

# Crear las variables de decisión
# x[(i, j)] es el flujo de tráfico en el enlace de i a j
x = pulp.LpVariable.dicts("x", enlaces, lowBound=0, cat='Continuous')

# Definir la función objetivo (minimizar costos operativos y de transmisión)
problem += pulp.lpSum([x[(i, j)] * enlaces[(i, j)]['costo'] for (i, j) in enlaces]), "Costo_Total"

# Definir las restricciones de capacidad de los enlaces
for (i, j) in enlaces:
    problem += x[(i, j)] <= enlaces[(i, j)]['capacidad'], f"Capacidad_Enlace_{i}_{j}"

# Definir las restricciones de balance de flujo en cada nodo
for nodo in nodos:
    flujo_entrante = pulp.lpSum([x[(i, j)] for (i, j) in enlaces if j == nodo])
    flujo_saliente = pulp.lpSum([x[(i, j)] for (i, j) in enlaces if i == nodo])

    demanda_entrante = pulp.lpSum([demanda for (origen, destino), demanda in demandas.items() if destino == nodo])
    demanda_saliente = pulp.lpSum([demanda for (origen, destino), demanda in demandas.items() if origen == nodo])

    problem += (flujo_entrante + demanda_saliente) == (flujo_saliente + demanda_entrante), f"Balance_Flujo_{nodo}"

# Resolver el problema
problem.solve()

# Imprimir los resultados
print("Estado:", pulp.LpStatus[problem.status])
print("Costo Total:", pulp.value(problem.objective))
for (i, j) in enlaces:
    print(f"Flujo en el enlace {i}->{j}: {pulp.value(x[(i, j)])} unidades")


Estado: Infeasible
Costo Total: 350.0
Flujo en el enlace A->B: 25.0 unidades
Flujo en el enlace A->C: 10.0 unidades
Flujo en el enlace B->C: 40.0 unidades
Flujo en el enlace B->D: 10.0 unidades
Flujo en el enlace C->D: 35.0 unidades


Escenario 1: Alta capacidad y baja demanda
En este escenario, los enlaces tienen alta capacidad y las demandas de tráfico son relativamente bajas.

enlaces_escenario1 = {
    ('A', 'B'): {'capacidad': 50, 'costo': 2},
    ('A', 'C'): {'capacidad': 40, 'costo': 3},
    ('B', 'C'): {'capacidad': 60, 'costo': 1},
    ('B', 'D'): {'capacidad': 30, 'costo': 2},
    ('C', 'D'): {'capacidad': 70, 'costo': 1}
}

demandas_escenario1 = {
    ('A', 'D'): 10,
    ('B', 'D'): 20,
    ('A', 'C'): 5
}


Escenario 2: Baja capacidad y alta demanda
En este escenario, los enlaces tienen baja capacidad y las demandas de tráfico son relativamente altas.

enlaces_escenario2 = {
    ('A', 'B'): {'capacidad': 15, 'costo': 3},
    ('A', 'C'): {'capacidad': 10, 'costo': 5},
    ('B', 'C'): {'capacidad': 20, 'costo': 2},
    ('B', 'D'): {'capacidad': 10, 'costo': 4},
    ('C', 'D'): {'capacidad': 25, 'costo': 3}
}

demandas_escenario2 = {
    ('A', 'D'): 20,
    ('B', 'D'): 25,
    ('A', 'C'): 15
}


Escenario 3: Balanceado
En este escenario, las capacidades y las demandas están balanceadas.

enlaces_escenario3 = {
    ('A', 'B'): {'capacidad': 25, 'costo': 2},
    ('A', 'C'): {'capacidad': 20, 'costo': 3},
    ('B', 'C'): {'capacidad': 30, 'costo': 1},
    ('B', 'D'): {'capacidad': 20, 'costo': 2},
    ('C', 'D'): {'capacidad': 35, 'costo': 1}
}

demandas_escenario3 = {
    ('A', 'D'): 15,
    ('B', 'D'): 15,
    ('A', 'C'): 10
}
